# 쇼핑몰_훈스_신상크롤링

### 자동으로 csv 파일로 저장

In [1]:
import os 
import sys
import requests
from bs4 import BeautifulSoup
import pandas as pd                      # 가장 기초적인 데이터전처리 패키지
import re                                # 정규표현식을 사용할 수 있는 패키지
from datetime import datetime            # 날짜를 표시해주는 패키지

In [2]:
base_url = 'http://mr-s.co.kr'

In [3]:
data = pd.DataFrame({"date" : [],
                    "image" : [],
                    "name" : [],
                    "price" : [],
                    "link" : []})

req = requests.get(base_url)
if req == 200 :
    soup = BeautifulSoup(req.content, 'html.parser')
else :
    req = requests.get(base_url, headers = {'user-agent': ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)'
                                                                            'Chrome/71.0.3578.98 Safari/537.36')})
    soup = BeautifulSoup(req.content, 'html.parser')

category = soup.find_all('li', {'class' :'red'}) # 카테고리를 구성하는 클래스명이 훈스샵과 다름, 확인해야함

for category_name in category :
    cn = category_name.text  # 해당 텍스트가 img 태그의 alt 에 있을 수도 있고, 그냥 텍스트로 존재할 수 있다.
    if 'new' in cn.lower() :
        new_category = category_name

new = new_category.find('a')['href']
new_url = base_url + new

# new 카테고리 접근 후 데이터 추출
new_req = requests.get(new_url)
if new_req == 200 :
    soup1 = BeautifulSoup(new_req.content, 'html.parser')
else :
    new_req = requests.get(new_url, headers = {'user-agent': ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)'
                                                                            'Chrome/71.0.3578.98 Safari/537.36')})
    soup1 = BeautifulSoup(new_req.content, 'html.parser')

page_source = soup1.find('div',{'class' :'xans-element- xans-product xans-product-normalpaging'}) # 페이지를 구성하는 클래스명이 훈스샵과 다르다. 확인해야함
page_list = page_source.find_all('a')
page_last = page_list[len(page_list)-1]['href']
page_num = page_last[len(page_last)-1:]

for page in range(1,int(page_num)+1) :
    new_page = new_url + '&page=' + '{}'

    np = requests.get(new_page.format(str(page)))
    if np == 200 :
        soup2 = BeautifulSoup(np.content, 'html.parser')
    else :
        np = requests.get(new_page.format(str(page)), headers = {'user-agent': ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)'
                                                                            'Chrome/71.0.3578.98 Safari/537.36')})
        soup2 = BeautifulSoup(np.content, 'html.parser')

    products = soup2.find_all('li', {'class' : 'item xans-record-'}) # 각각의 상품을 구성하는 클래스명이 훈스샵과 다르다. 확인해야함.

    for product in products :
        try : # 에러가 있는 상품을 넘겨뛰는, 에러발생 시 제외하는 try문
            image = product.find('div', {'class' : 'box'})
            image_url = base_url + image.find('img')['src'] # 이미지가 하나로 구성된것도 있으며, 여러개가 있는 것이 있다.
                                                            # 다만, 여러가지 이미지가 번갈아가며 표시되어야 하는데, 이것을 구성하는 방식을 몰라 하나만 추출

            name = product.find('p', {'class' : 'name'}).find('span').text
            name = re.sub('[^가-힣0-9a-zA-Z\\s\+\[\]\,\.\=\-\_\~\!\@\#\$\%\^\&\*\(\)\;\:\?]','',name) # 유니코드로 변환하기 어려운 특수문자가 있는 경우 글자가 깨진다.
                                                                                                      # 따라서, 필요한 글자들만 정규표현식으로 표시해준다.

            price_0 = product.find('ul', {'class' : 'xans-element- xans-product xans-product-listitem'}) # 가격정보가 있는 곳의 클래스
            price_1 = price_0.find('span', {'style' : 'font-size:16px;color:#000000;font-weight:bold;'}) # 원 가격, 할인가격 등 다양한 가격이 있는데, 할인가격으로 표시해주는 것
            price_2 = price_1.text

            link = product.find('div', {'class' : 'box'}).find('a')['href']
            link_url = base_url + link

            insert_data = pd.DataFrame({"date" : datetime.today().strftime("%Y-%m-%d"),
                            "image" : [image_url],
                            "name" : [name],
                            "price" : [price_2],
                            "link" : [link_url]})

            data = data.append(insert_data)
            
        except :
            pass

data = data.drop_duplicates(subset='image', keep='last') # 중복된 부분이 있다면 가장 최근것으로 대체
data.index = range(len(data))

try :
    pred_data = pd.read_csv('mr-s.csv', encoding='cp949') # 이전에 있던 파일을 포함해 조정하는 것
    all_data = pred_data.append(data)
    data = all_data.drop_duplicates(subset='image', keep='last')
    data.index = range(len(data))
except :
    pass

data.to_csv(os.getcwd() + "/" + "mr-s" + ".csv", mode = 'w', header = True, index = None, encoding='euc_kr') # 인코딩 변경


In [4]:
data

,date,image,name,price,link
0,2019-01-25,http://mr-s.co.kr//mr-s.co.kr/web/product/tiny...,클레이 20수박스싱글긴팔,"9,800원",http://mr-s.co.kr/product/detail.html?product_...
1,2019-01-25,http://mr-s.co.kr//mr-s.co.kr/web/product/tiny...,[상의1+1] 2019 설랜박스,"7,800원",http://mr-s.co.kr/product/detail.html?product_...
2,2019-01-25,http://mr-s.co.kr//mr-s.co.kr/web/product/tiny...,위클리 후드 반패딩,"24,800원",http://mr-s.co.kr/product/detail.html?product_...
3,2019-01-25,http://mr-s.co.kr//mr-s.co.kr/web/product/tiny...,알래스카 기모 맨투맨,"19,800원",http://mr-s.co.kr/product/detail.html?product_...
4,2019-01-25,http://mr-s.co.kr//mr-s.co.kr/web/product/tiny...,비체 더블 양털 누빔 코트,"57,800원",http://mr-s.co.kr/product/detail.html?product_...
5,2019-01-25,http://mr-s.co.kr//mr-s.co.kr/web/product/tiny...,원블 라인 배색 기모 후드티,"18,800원",http://mr-s.co.kr/product/detail.html?product_...
6,2019-01-25,http://mr-s.co.kr//mr-s.co.kr/web/product/tiny...,브리티시 반목 니트지 맨투맨,"13,800원",http://mr-s.co.kr/product/detail.html?product_...
7,2019-01-25,http://mr-s.co.kr//mr-s.co.kr/web/product/tiny...,어라운드 기모 후드티,"24,800원",http://mr-s.co.kr/product/detail.html?product_...
8,2019-01-25,http://mr-s.co.kr//mr-s.co.kr/web/product/tiny...,하이퀄 웰론 숏패딩,"39,800원",http://mr-s.co.kr/product/detail.html?product_...
9,2019-01-25,http://mr-s.co.kr//mr-s.co.kr/web/product/tiny...,미주리 골지 폴라티,"12,800원",http://mr-s.co.kr/product/detail.html?product_...
